# Ingest an existing Dataset or Model

In this notebook we are going to showcase how to ingest an existing dataset or model into EOTDL.

Once it is ingested, you can use it in the same way as any other dataset or model in EOTDL (exploring, downloading, etc.).

## Ingesting through the CLI

The recommended way to ingest a dataset is using the CLI.

In [1]:
!eotdl datasets ingest --help

                                                                                
 Usage: eotdl datasets ingest [OPTIONS]                                         
                                                                                
 Ingest a dataset to the EOTDL.                                                 
 This command ingests the dataset to the EOTDL. The dataset must be a folder    
 with the dataset files, and at least a README.md file (and a catalog.json file 
 for Q1+). If these files are missing, the ingestion will not work. All the     
 files in the folder will be uploaded to the EOTDL.                             
                                                                                
 The following constraints apply to the dataset name:                           
 - It must be unique                                                            
 - It must be between 3 and 45 characters long                                  
 - It can only contain alpha

In order to ingest a dataset you will need a folder in your system with the data you want to upload.

In [3]:
!ls workshop_data

EuroSAT-small


For this tutorial we are going to work with a subsample of the [EuroSAT](https://www.eotdl.com/datasets/EuroSAT-RGB) dataset.

In [5]:
from glob import glob 

files = glob('workshop_data/EuroSAT-small/**/*.*', recursive=True)
files

['workshop_data/EuroSAT-small/README.md',
 'workshop_data/EuroSAT-small/Forest/Forest_1.tif',
 'workshop_data/EuroSAT-small/Forest/Forest_2.tif',
 'workshop_data/EuroSAT-small/Forest/Forest_3.tif',
 'workshop_data/EuroSAT-small/AnnualCrop/AnnualCrop_1.tif',
 'workshop_data/EuroSAT-small/AnnualCrop/AnnualCrop_3.tif',
 'workshop_data/EuroSAT-small/AnnualCrop/AnnualCrop_2.tif']

A `README.md` file is required for Q0 datasets and models, containing some basic required information (dataset authors, licens, link to source and dataset name).

In [6]:
!cat workshop_data/EuroSAT-small/README.md

---
authors:
- Patrick Helber
license: open
source: http://madm.dfki.de/downloads
name: EuroSAT-small
---

# EuroSAT small

This is a test datasets used for the PhiLab24 workshop. It is a subset of the EuroSAT dataset.

The chosen name is the one that will appear in the repository, hence it must be unique, between 3 and 45 characters long and can only contain alphanumeric characters and dashes (learn more at [https://www.eotdl.com/docs/datasets/ingest](https://www.eotdl.com/docs/datasets/ingest)).

Trying to ingest a dataset without a `README.md` file will fail.

If everything is correct, the ingestion process should suceed.

In [7]:
!eotdl datasets ingest -p workshop_data/EuroSAT-small/

Uploading directory workshop_data/EuroSAT-small...
generating list of files to upload...
100%|███████████████████████████████████████████| 6/6 [00:00<00:00, 4246.68it/s]
6 new files will be ingested
0 files already exist in dataset
0 large files will be ingested separately
New version created, version: 1
generating batches...
100%|█████████████████████████████████████████| 6/6 [00:00<00:00, 162360.15it/s]
Uploading 6 small files in 1 batches...
Uploading batches: 100%|█████████████████████| 1/1 [00:00<00:00,  1.47batches/s]


And now your dataset is avilable at EOTDL

In [8]:
!eotdl datasets list -n eurosat-small

['EuroSAT-small']


> Since the `EuroSAT-small` name is already taken, this process should fail for you. To solve it, just upload the dataset with a different name. However, this will polute the EOTDL with test datasets so we encourage you to try the ingestion process with a real dataset that you want to ingest (or overwrite your test dataset in the future with useful data).

In order to ingest Q1+ datasets, a valid STAC catalog is required instead of the `README.md` file. We will explore this in the [data curation](tutorials/workshops/bids23/05_STAC_metadata.ipynb) notebook.

Let's now ingest the model that we trained in the previous notebook.

In [9]:
!ls data/*.onnx

data/model.onnx


In [10]:
!mkdir -p data/EuroSAT-RGB-PhiLab24-model
!cp data/model.onnx data/EuroSAT-RGB-PhiLab24-model/model.onnx

First, we need to create a folder with the model and the `README.md` file (and any other file that you want).

In [10]:
import yaml

metadata = {
	"name": "EuroSAT-RGB-BiDS23",
	"authors": ["Juan B. Pedro"],
	"license": "open",
	"source": "https://github.com/earthpulse/eotdl/blob/develop/tutorials/workshops/bids23/02_training.ipynb"
}

with open('data/EuroSAT-RGB-BiDS-model/metadata.yml', 'w') as outfile:
	yaml.dump(metadata, outfile, default_flow_style=False)

In [12]:
text = """---
name: EuroSAT-RGB-PhiLab24
authors: 
  - Juan B. Pedro
license: open
source: https://github.com/earthpulse/eotdl/blob/develop/tutorials/workshops/philab24/02_training.ipynb
---

# EuroSAT-RGB-PhiLab24

This is a toy model trained with the EuroSAT dataset for the PhiLab24 workshop.
"""

with open('data/EuroSAT-RGB-PhiLab24-model/README.md', 'w') as outfile:
	outfile.write(text)

Now we can ingest the model to EOTDL

In [13]:
!eotdl models ingest -p data/EuroSAT-RGB-PhiLab24-model

Uploading directory data/EuroSAT-RGB-PhiLab24-model...
generating list of files to upload...
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 17.36it/s]
1 new files will be ingested
0 files already exist in dataset
1 large files will be ingested separately
New version created, version: 1
ingesting large files...
ingesting file: model.onnx
models
42.65/42.65 MB: : 5it [00:03,  1.26it/s]                                        


## Versioning

By default, every time you re-upload a dataset or model a new version is created. We apply versioning at dataset/model and file level, meaning only new or modified files will be uploaded in future re-uploads, downloading the appropriate files for each version.

In [19]:
!echo "hello" > workshop_data/EuroSAT-small/hello.txt
!eotdl datasets ingest -p workshop_data/EuroSAT-small/

Uploading directory workshop_data/EuroSAT-small...
generating list of files to upload...
100%|███████████████████████████████████████████| 7/7 [00:00<00:00, 3436.75it/s]
1 new files will be ingested
6 files already exist in dataset
0 large files will be ingested separately
New version created, version: 2
generating batches...
100%|██████████████████████████████████████████| 1/1 [00:00<00:00, 14563.56it/s]
Uploading 1 small files in 1 batches...
Ingesting existing files: 100%|██████████████| 1/1 [00:00<00:00,  3.61batches/s]


When you download a dataset, the latest version is used by default.

In [20]:
!eotdl datasets get EuroSAT-small

100%|███████████████████████████████████████████| 7/7 [00:02<00:00,  2.46file/s]
Data available at /home/juan/.cache/eotdl/datasets/EuroSAT-small/v2


However, you can specify the version

In [21]:
!eotdl datasets get EuroSAT-small -v 1

Dataset `EuroSAT-small v1` already exists at /home/juan/.cache/eotdl/datasets/EuroSAT-small/v1. To force download, use force=True or -f in the CLI.


In [22]:
!ls $HOME/.cache/eotdl/datasets/EuroSAT-small

v1  v2


You can explore the different versions in the user interface.

## Ingesting through the Library

You can ingest datasets and models using the library

In [28]:
from eotdl.datasets import ingest_dataset

try:
    ingest_dataset("workshop_data/EuroSAT-small")
except Exception as e:
    print(e)

Uploading directory workshop_data/EuroSAT-small...
generating list of files to upload...


100%|██████████| 7/7 [00:00<00:00, 3520.82it/s]

No new files to upload


In [29]:
from eotdl.models import ingest_model

try:
    ingest_model("data/EuroSAT-RGB-PhiLab24-model")
except Exception as e:
    print(e)

Uploading directory data/EuroSAT-RGB-PhiLab24-model...
generating list of files to upload...


100%|██████████| 1/1 [00:00<00:00, 16.56it/s]

No new files to upload


## Discussion and Contribution opportunities

Feel free to ask questions now (live or through Discord) and make suggestions for future improvements.

- What features concerning ingestion would you like to see?
- What other features concerning versioning would you like to see?